In [ ]:
# Installation Library

!pip install transformers datasets sentencepiece rouge 

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
dataset = load_dataset("billsum")

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
test_cases =  dataset['ca_test'] # California bills

In [ ]:
case1 = test_cases[1]['text']
sum1 =  test_cases[1]['summary']

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "murali-admin/bart-billsum-1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [ ]:
batch = tokenizer(case1, truncation=True, padding="longest", return_tensors="pt").to(device)

In [ ]:
batch

{'input_ids': tensor([[   0,  133,   82,  ..., 8257,    9,    2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [ ]:
summary1 = model.generate(**batch, max_length=2000, min_length=500)
systemsummary = tokenizer.batch_decode(summary1, skip_special_tokens=True)

In [ ]:
standard_summary = sum1
system_summary = systemsummary[0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, system_summary.split('.')[0])
pd.DataFrame(score[0]).set_index([['recall','precision','f-measure']])

,rouge-1,rouge-2,rouge-l
recall,1.000000,1.000000,1.000000
precision,0.398693,0.338645,0.398693
f-measure,0.570093,0.505952,0.570093


Test DataSet

In [ ]:
CasesText = test_cases['text']
GoldSummary = test_cases['summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1237, 1237)

In [ ]:
# CasesText[:500]
# CasesText[500:800]
# CasesText[800:]

In [ ]:
SystemSummary = []

for i, case in enumerate(CasesText[500:800]):
    
    batch = tokenizer(case, truncation=True, padding="longest", return_tensors="pt").to(device)  
    summary = model.generate(**batch, max_length=3000, min_length=500)
    Summary = tokenizer.batch_decode(summary, skip_special_tokens=True)

    SystemSummary.append(Summary)
    print(i)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary[500:800], SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

Summaries.to_csv(path + "BartBillSum2.csv", header=True, index=False)

**BART Model Score on Test**

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

In [ ]:
file1 = pd.read_csv(path + "BartBillSum1.csv")
file2 = pd.read_csv(path + "BartBillSum2.csv")
file3 = pd.read_csv(path + "BartBillSum3.csv")

In [ ]:
file1.shape, file2.shape, file3.shape, 

((500, 2), (300, 2), (437, 2))

In [ ]:
BartSum = pd.concat([file1, file2, file3])

In [ ]:
BartSum.reset_index(inplace= True, drop=True)

In [ ]:
BartSum.sample(5)

,GoldSummary,SystemSummary
727,Existing law provides for the Medi-Cal program...,This bill directs the Department of Health and...
139,(1) The California Community Care Facilities A...,AmAmends the Health and Safety Code to prohibi...
1076,Existing law provides generally for procedures...,AmAmends the Code of Civil Procedure to permit...
613,"(1) Existing law, the San Francisco Bay Restor...",Authorizes the authority to levy a benefit ass...
916,Existing law requires the State Department of ...,Amends the Welfare and Institutions Code to re...


In [ ]:
system_summaryUp = [''.join(i.split('.')[:2]) for i in BartSum['SystemSummary']]

In [ ]:
BartSum['system_summaryUp'] = system_summaryUp

In [ ]:
system_summary = BartSum['SystemSummary']
standard_summary = BartSum['GoldSummary']
system_summaryUp = BartSum['system_summaryUp']

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summaryUp, standard_summary, avg=True)

In [ ]:
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])

In [ ]:
BartRouge

,rouge-1,rouge-2,rouge-l
recall,0.208918,0.091433,0.176003
precision,0.433413,0.228846,0.370649
f-measure,0.260234,0.118470,0.220206


In [ ]:
path = "/content/drive/MyDrive/LegSuM/scores/"

BartRouge.to_csv(path + "BartRouge.csv", header=True)